# 作成したモデルのdeployとserving

このハンズオンでは、Vertex AIにモデルをデプロイし、推論APIとして利用できるようにしていきます。

先ほど作成した犬種/猫種判別モデルをデプロイすることで、その手順を確認していきましょう。

In [ ]:
%%bash
## <todo> 自分の名前を入力してください
USER=___ # 自分の名前
REGION=asia-northeast1
BUCKET=mixi-ml-handson-2023
MODEL=pet_model

gcloud ai models upload \
  --region=${REGION} \
  --display-name=oxford-pet-${USER} \
  --container-image-uri=image-uri=asia-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-11:latest \
  --artifact-uri=gs://${BUCKET}/${USER}/${MODEL}

gcloud ai models list \
  --region=${REGION} \
  --filter=oxford-pet-${USER}
    
gcloud ai endpoints create \
  --region=${REGION} \
  --display-name=oxford-pet-${USER}

上記コードの実行には数分の時間がかかります。  
実行が終わったら、出力された`MODEL_ID`と、endpointを作成した際に出力された`ENDPOINT_ID` `'・・・/asia-northeast1/endpoints/<ENDPOINT_ID>'`を使って、endpointにmodelをdeployします。

In [ ]:
%%bash
## <todo> ___ 部分を埋めてください
USER=___ # 自分の名前
REGION=asia-northeast1
MODEL_ID=___ # 前のセルで出力されたMODEL_ID
ENDPOINT_ID=___ # 前のセルで出力されたENDPOINT_ID

gcloud ai endpoints deploy-model ${ENDPOINT_ID} \
  --region=${REGION} \
  --model=${MODEL_ID} \
  --display-name=oxford-pet-${USER}


上記コードの実行には数分の時間がかかります。

versionの作成が完了したら、このendpointにリクエストを投げて結果が返ってくるか確認してみます。  
まず、02と同じようにデータセットを用意していきましょう。

In [ ]:
import tensorflow as tf
import numpy as np

image_size = (200, 200) # Vertex AIのサイズ制限上、意図的に224->200にしています
batch_size = 16
DATASET_DIR = "../02_transfer_learning/dataset"

## <todo> ___ を埋めて、02と同じ検証データセットが使えるようにしてください。
## このメソッドはdefaultでshuffle=Trueになっているので、同じデータを取得するにはseed値を同じにする必要があります。
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="validation",
    label_mode='categorical',
    seed=___,
    image_size=image_size,
    batch_size=batch_size,
)

データセットが用意できたら、endpointにリクエストを投げて結果を受け取るメソッドを作成します。

In [ ]:
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

# <todo> ENDPOINT_IDを埋めてください
PROJECT_ID = 'hr-mixi'
ENDPOINT_ID = '___' # 出力されたENDPOINT_ID
REGION = "asia-northeast1"

def predict_json(instances):    
    aiplatform.init(project=PROJECT_ID, location=REGION)
    endpoint = aiplatform.Endpoint(ENDPOINT_ID)
    instances = [json_format.ParseDict(s, Value()) for s in instances]
    response = endpoint.predict(instances=instances)
    return response.predictions

これで結果を受け取る部分はできたので、02で作成したpredict_datasetメソッドのresult部分を、作成したpredict_jsonメソッドに変えてみましょう。


In [ ]:
def predict_dataset():
    for images, labels in val_data.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            image = images[i].numpy()
            plt.imshow(image.astype("uint8"))
            norm_img = image / 255.0
            norm_img = np.expand_dims(norm_img, 0)
            norm_img = norm_img.tolist()
            ## <todo> ___ を埋めて、予測を受け取れるようにしてください。
            result =  ___(___)
            plt.title(
                "label:" + val_data.class_names[np.argmax(labels[i])] +
                "\npredict:" + val_data.class_names[np.argmax(result[0])]
            )
        plt.axis("off")

In [ ]:
import matplotlib.pyplot as plt
labels = val_data.class_names

plt.figure(figsize=(10, 12))
predict_dataset()

これで、modelの部分を推論APIに置き換えることができました。

## Gradioを使って犬種/猫種判別を行うデモアプリを作成する

model部分のAPI化ができたので、機械学習を使った簡単なデモアプリを作ってみましょう。

まず、お手軽にML系のwebアプリケーションが作成できるgradioをinstallします。

In [ ]:
!pip install -q --user gradio

In [ ]:
import gradio as gr

上記ImportErrorがでた場合は、kernelをrestartして、<b>bashコマンド以降のセルを改めて実行してみてください。</b>

完了したら、デモアプリを作成してみましょう。  
コード実行ができたら、public URLを確認してみてください。

In [ ]:
labels = val_data.class_names

def predict(image):
    image = image / 255.0
    image = np.expand_dims(image, 0)    
    image = image.tolist()
    pred = predict_json(image)[0]
    confidences = {labels[i]: pred[i] for i in range(len(labels))}    
    return confidences

demo = gr.Interface(predict, gr.Image(shape=(200, 200)), outputs=gr.outputs.Label(num_top_classes=5))

# <todo> ""部分に独自のuserとpasswordを入力してください。
# share=Trueになっていると、public URLとして公開されるため、必ず各自でpassを設定して入るようにしてください。
demo.launch(share=True, auth=("", ""))

犬種/猫種判定アプリが作れていましたでしょうか。  
データセットにない犬、猫などは、  
犬は、https://www.min-inuzukan.com/  
猫は、https://www.min-nekozukan.com/  
にあるので、これらを参考にしつつ正しく判別できるか試してみてください。 